## Connection to table in GDrive

In [1]:
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
import gspread
import warnings
warnings.filterwarnings("ignore")

---
### Extracted table from automatic sources

In [2]:
table1 = pd.read_csv('../datasets/Arxiv_results.csv', header=None).rename(columns={0:'url',1:'description',2:'topic'})
table1['source'] = 'Arxiv'
table1['source_type'] = 'Research'
table2 = pd.read_csv('../datasets/DBLP_results.csv', header=None).rename(columns={0:'url',1:'description',2:'topic'})
table2['source'] = 'DBLP'
table2['source_type'] = 'Research'
table3 = pd.read_csv('../datasets/Gitlab_results.csv', header=None).rename(columns={0:'url',1:'description',2:'topic'})
table3['source'] = 'Gitlab'
table3['source_type'] = 'Software registry'
table4 = pd.read_csv('../datasets/Googleplay_results.csv', header=None).rename(columns={0:'url',1:'description',2:'topic'})
table4['source'] = 'Google play'
table4['source_type'] = 'Software registry'
table5 = pd.read_csv('../datasets/Googlescholar_results.csv', header=None).rename(columns={0:'url',1:'description',2:'topic'})
table5['source'] = 'Google scholar'
table5['source_type'] = 'Research'
table6 = pd.read_excel('../datasets/github_monitoring_tools.xlsx')[['url','description','topic']]
table6['source'] = 'Github'
table6['source_type'] = 'Software registry'
table7 = pd.read_excel('../datasets/patents_monitoring_tools.xlsx')[['url','description','topic']]
table7['source'] = 'EU Patents Database'
table7['source_type'] = 'Software registry'

In [3]:
# Merge the tables
tableA = table1.append(table2, sort=True).reset_index()
tableB = tableA.append(table3, sort=True).reset_index()[['url','description','topic','source','source_type']]
tableC = tableB.append(table4, sort=True).reset_index()[['url','description','topic','source','source_type']]
tableD = tableC.append(table5, sort=True).reset_index()[['url','description','topic','source','source_type']]
tableE = tableD.append(table6, sort=True).reset_index()[['url','description','topic','source','source_type']]
tableF = tableE.append(table7, sort=True).reset_index()[['url','description','topic','source','source_type']]

In [4]:
automathic_table = tableF
automathic_table['name'] = 'NA'
automathic_table['method'] = 'Crawler search'

In [5]:
automathic_table.head()

,url,description,topic,source,source_type,name,method
0,http://arxiv.org/abs/1709.09302v3,We consider a market in which capacity-constra...,market monitoring,Arxiv,Research,NA,Crawler search
1,http://arxiv.org/abs/cs/0306024v1,The DESY Computer Center is the home of O(1000...,market monitoring,Arxiv,Research,NA,Crawler search
2,http://arxiv.org/abs/1607.03583v1,This paper analyzes repeated multimarket conta...,market monitoring,Arxiv,Research,NA,Crawler search
3,http://arxiv.org/abs/1809.01640v1,The paper presents web based information syste...,market monitoring,Arxiv,Research,NA,Crawler search
4,http://arxiv.org/abs/1602.07063v2,This article investigates graph analysis for i...,market monitoring,Arxiv,Research,NA,Crawler search


---
### Connect with the table form manual search

In [6]:
## Connect to our service account
project = 
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name(project, scope)
gc = gspread.authorize(credentials)

In [7]:
##Get candidate data sheet from Google Drive
spreadsheet_key = 
worksheet_name = 'manual_search'

In [8]:
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet(worksheet_name)
data = worksheet.get_all_values()
manual_table = pd.DataFrame(data[1:], columns=data[0])

---
### Merge the tables and remove duplictes

In [9]:
manual_table.drop(columns='comment', inplace=True)
manual_table.head(2)

,name,url,description,topic,method,source,source_type
0,Detection of Zombie PCs Based on Email Spam An...,https://apps.webofknowledge.com/full_record.do...,we propose a system that detects botnets and z...,"""IP blocking""",Research search,Web of Science Database,Research
1,Engineering an Agent-based System for Product ...,https://users.isc.tuc.gr/~nispanoudakis/resour...,For developing pricing by companies,"""individual pricing""",Research search,Web of Science Database,Research


In [10]:
len(manual_table)

72

In [11]:
automathic_table.head(2)

,url,description,topic,source,source_type,name,method
0,http://arxiv.org/abs/1709.09302v3,We consider a market in which capacity-constra...,market monitoring,Arxiv,Research,NA,Crawler search
1,http://arxiv.org/abs/cs/0306024v1,The DESY Computer Center is the home of O(1000...,market monitoring,Arxiv,Research,NA,Crawler search


In [12]:
len(automathic_table)

4276

In [13]:
#just taking into account the automatic table
table = automathic_table
#table = manual_table.append(
#    automathic_table, sort=True).reset_index()[['url','name','description','topic','method','source','source_type']]

In [14]:
#duplicates identification
list_id = list(table.url)
list_uniques = list(table.url.unique())
print('Total tools: {}\nUnique tools: {}\n======='.format(len(list_id),len(list_uniques)))
for i in range(0,10):
    sets = list(set([x for x in list_id if list_id.count(x) > i]))
    print('url with {} duplicates: {}'.format(i, len(sets)))
print('=======\nlines to remove: {}'.format(len(list_id) - len(list_uniques)))

Total tools: 4276
Unique tools: 3529
url with 0 duplicates: 3529
url with 1 duplicates: 373
url with 2 duplicates: 158
url with 3 duplicates: 87
url with 4 duplicates: 55
url with 5 duplicates: 33
url with 6 duplicates: 23
url with 7 duplicates: 13
url with 8 duplicates: 3
url with 9 duplicates: 1
lines to remove: 747


In [15]:
## Save the data back to a new sheet in the dataframe
table = table.drop_duplicates(subset ='url')

In [16]:
table['topic'] = [row.replace("'","") for row in table.topic]

In [17]:
len(table)

3529

In [18]:
table.head()

,url,description,topic,source,source_type,name,method
0,http://arxiv.org/abs/1709.09302v3,We consider a market in which capacity-constra...,market monitoring,Arxiv,Research,NA,Crawler search
1,http://arxiv.org/abs/cs/0306024v1,The DESY Computer Center is the home of O(1000...,market monitoring,Arxiv,Research,NA,Crawler search
2,http://arxiv.org/abs/1607.03583v1,This paper analyzes repeated multimarket conta...,market monitoring,Arxiv,Research,NA,Crawler search
3,http://arxiv.org/abs/1809.01640v1,The paper presents web based information syste...,market monitoring,Arxiv,Research,NA,Crawler search
4,http://arxiv.org/abs/1602.07063v2,This article investigates graph analysis for i...,market monitoring,Arxiv,Research,NA,Crawler search


---
## Sampling

In [19]:
def sampleSize(population_size, margin_error=.05, confidence_level=.99, sigma=1/2):
    alpha = 1 - (confidence_level)
    # dictionary of confidence levels and corresponding z-scores
    # computed via norm.ppf(1 - (alpha/2)), where norm is
    zdict = {
        .90: 1.645,
        .91: 1.695,
        .99: 2.576,
        .97: 2.17,
        .94: 1.881,
        .93: 1.812,
        .95: 1.96,
        .98: 2.326,
        .96: 2.054,
        .92: 1.751
    }
    if confidence_level in zdict:
        z = zdict[confidence_level]
    else:
        from scipy.stats import norm
        z = norm.ppf(1 - (alpha/2))
    N = population_size
    M = margin_error
    a = z**2 * sigma**2 * (N / (N-1))
    b = M**2 + ((z**2 * sigma**2)/(N-1))
    return int(round(a/b,0))

In [20]:
sample_size = sampleSize(population_size=len(table),
                         margin_error=.1, confidence_level=.95, sigma=1/2)
print('Sample Size: {}'.format(sample_size))

Sample Size: 94


In [21]:
catalogue = pd.read_csv('catalogue.csv').fillna(method='ffill')
catalogue.columns = ['group','desciption','subgroup','source','terms','keywords','datasource','comments','-','-+']
catalogue = catalogue[['group','subgroup','keywords']]
catalogue.head(3)

,group,subgroup,keywords
0,Advertising,market monitoring,market monitoring
1,Advertising,behavioural tracking,consumer profiling algorithm
2,Advertising,behavioural tracking,behavioral profiling


In [22]:
table_augmented = pd.merge(table, catalogue, left_on='topic', right_on='keywords', how='left')
table_augmented['stratum'] = table_augmented['source'] + ' - ' + table_augmented['group']
#table_augmented['stratum'] = table_augmented['group']

In [23]:
groups = table_augmented.groupby('group').count()['url'].reset_index()
groups['proportion'] = groups.url/groups.url.sum()
groups['selection'] = [int(i) for i in groups.proportion*sample_size]

In [24]:
groups

,group,url,proportion,selection
0,Advertising,1773,0.502409,47
1,Compliance,77,0.021819,2
2,Failure of Web accessibility,180,0.051006,4
3,Geoblocking,57,0.016152,1
4,Price Transparency Issues,235,0.066591,6
5,Scam (Online Fraud),875,0.247946,23
6,Scarcity cues,40,0.011335,1
7,Unfair terms issues,292,0.082743,7


In [25]:
stratums = table_augmented.groupby('stratum').count()['url'].reset_index()
stratums['proportion'] = stratums.url/stratums.url.sum()
stratums['selection'] = [int(i) for i in stratums.proportion*sample_size]

In [26]:
stratums

,stratum,url,proportion,selection
0,Arxiv - Advertising,370,0.104846,9
1,Arxiv - Compliance,74,0.020969,1
2,Arxiv - Failure of Web accessibility,57,0.016152,1
3,Arxiv - Geoblocking,24,0.006801,0
4,Arxiv - Price Transparency Issues,110,0.031170,2
5,Arxiv - Scam (Online Fraud),317,0.089827,8
6,Arxiv - Scarcity cues,33,0.009351,0
7,Arxiv - Unfair terms issues,100,0.028337,2
8,DBLP - Advertising,314,0.088977,8
9,DBLP - Compliance,2,0.000567,0


In [27]:
np.random.seed(32)
matrix = []
for source in stratums.stratum:
    range_group = table_augmented[table_augmented.stratum == str(source)].index
    selection = stratums[stratums.stratum == str(source)].selection.values[0]
    #print(source, ' ', selection)
    group_sample = list(np.random.choice(range_group, selection))
    matrix.append(group_sample)
stratified_sample = sorted(set([item for sublist in matrix for item in sublist]))

In [28]:
table_augmented['first_sample'] = [1 if i in stratified_sample else 0 for i in table_augmented.index]        

In [29]:
table_augmented['first_sample'].sum()

74

In [30]:
#these are going to be excluded in the nex round of sam[ling]
index_first_sample = table_augmented[table_augmented['first_sample'] == 1].index

In [31]:
index_first_sample

Int64Index([  43,   71,   88,   89,  215,  234,  252,  259,  310,  462,  469,
             492,  509,  527,  565,  686,  707,  801,  813,  848,  912,  982,
            1020, 1137, 1147, 1162, 1205, 1242, 1251, 1334, 1352, 1498, 1514,
            1538, 1543, 1632, 1664, 1753, 1828, 1852, 1873, 1966, 2028, 2114,
            2117, 2131, 2209, 2227, 2298, 2361, 2467, 2498, 2533, 2548, 2622,
            2639, 2678, 2877, 2884, 2931, 2952, 2957, 2982, 3014, 3078, 3197,
            3265, 3289, 3328, 3391, 3475, 3479, 3507, 3512],
           dtype='int64')

---

In [32]:
wks_name = 'clean_database'
d2g.upload(table_augmented, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

<Worksheet 'clean_database' id:1709523549>

In [33]:
#d2g.upload(table_augmented[['topic','source']].drop_duplicates(subset ='topic'), spreadsheet_key,
#           'terms', credentials=credentials, row_names=False)

In [34]:
len(table_augmented)

3534